In [1]:
import cv2 
import os 

In [ ]:
# Scrape all folders with video in them
cwd = os.getcwd()
data_path = os.path.dirname(os.path.dirname(cwd)) + "\\Data\\data_sorted\\data"

for file in os.listdir(data_path):
    video_name = str.removesuffix(file, ".mp4")
    print(os.path.join(data_path,video_name))

In [ ]:
# Turn videos into frames and save them into folders, for each video.

cwd = os.getcwd()
data_path = os.path.dirname(os.path.dirname(cwd)) + "\\Data\\data_sorted\\data"
frames_path = os.path.dirname(os.path.dirname(cwd)) + "\\Data\\data_sorted\\frames"
os.mkdir(frames_path)

for file in os.listdir(data_path):

	cam = cv2.VideoCapture(os.path.join(data_path,file)) 
	video_name = str.removesuffix(file, ".mp4")

	try: 
		
		if not os.path.exists(os.path.join(frames_path,video_name)): 
			os.mkdir(os.path.join(frames_path,video_name)) 

	# if not created then raise error 
	except OSError: 
		print ('Error: Creating directory of data') 

	# frame 
	currentframe = 0

	while(True): 
		
		# reading from frame 
		ret,frame = cam.read() 

		if ret: 
			# if video is still left continue creating images 
			name = os.path.join(frames_path,video_name) + '\\' + str(currentframe).zfill(6) + '.PNG'
			print ('Creating...' + name) 

			# writing the extracted images 
			cv2.imwrite(name, frame) 

			# increasing counter so that it will 
			# show how many frames are created 
			currentframe += 1
		else: 
			break

	# Release all space and windows once done 
	cam.release() 


In [ ]:
frames_path = os.path.dirname(os.path.dirname(cwd)) + "\\Data\\data_sorted\\frames"

for dir in os.listdir(frames_path):

    dir_path = os.path.join(frames_path,dir)
    print(dir_path)

    for file in os.listdir(dir_path):
        name = "frame_"+str(file)
        os.rename(os.path.join(dir_path,file), os.path.join(dir_path,name))

In [1]:
from pycocotools.coco import COCO
import os 
import pandas as pd

In [2]:
path_base = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'Data', 'data_sorted')

coco_annotations_file_root = os.path.join(path_base,'labels', 'COCO')
coco_images_dir_root = os.path.join(path_base, 'frames')

In [ ]:
dict_list = []

for j in os.listdir(coco_images_dir_root):
    
    coco_annotations_file = os.path.join(coco_annotations_file_root, j+'.json')

    coco = COCO(coco_annotations_file)
    
    count_dict = {}

    classes = []
    for x in coco.cats:
        classes.append(coco.cats[x]['name'])
    
    for x in classes:
        catIds = coco.getCatIds(catNms=x) 
        imgIds = coco.getImgIds(catIds=catIds)
        count_dict[x] =  len(imgIds)

    dict_list.append(count_dict)

In [4]:
def merge_keys(ds):
    keys = set.union(*map(set, ds))
    return keys

In [5]:
keys = merge_keys(dict_list)

In [ ]:
keys

In [7]:
sum_dict = dict.fromkeys(keys, 0)

In [8]:
df = pd.DataFrame(columns=list(keys))

for x in dict_list:
    df.loc[len(df)] = x
    for y in x:
        sum_dict[y] += x[y]
df.loc[len(df)] = sum_dict

In [11]:
df.to_excel(os.path.dirname(os.path.dirname(cwd)) + "\\Data\\data_sorted\\labels\\count.xlsx")

In [ ]:
sum(sum_dict.values())